In [1]:
# Libary Imports
import sys
import torch
import random
import logging
import numpy as np
import geopandas as gpd
import matplotlib as plt

# Project Imports
from src.utils.config_loader import load_project_config
from src.graph_building.graph_construction import build_mesh, \
    define_catchment_polygon
from src.visualisation.mapped_visualisations import plot_interactive_mesh

In [2]:
# Set up logger config
logging.basicConfig(
    level=logging.INFO,
   format='%(levelname)s - %(message)s',
#    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)

# Set up logger for file and load config file for paths and params
logger = logging.getLogger(__name__)
config = load_project_config(config_path="config/project_config.yaml")
notebook = True

# Set up seeding to define global states
random_seed = config["global"]["pipeline_settings"]["random_seed"]
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Define notebook demo catchment
catchments_to_process = config["global"]["pipeline_settings"]["catchments_to_process"]
catchment = catchments_to_process[0]
run_defra_API_calls = config["global"]["pipeline_settings"]["run_defra_api"]  # True to run API calls

logger.info(f"Show Notebook Outputs: {notebook}")
logger.info(f"Notebook Demo Catchment: {catchment.capitalize()}")

INFO - Loading configuration from: /Users/charlotte/Desktop/Dissertation_Code/config/project_config.yaml
INFO - Show Notebook Outputs: True
INFO - Notebook Demo Catchment: Eden


In [3]:
define_catchment_polygon(
    england_catchment_gdf_path=config[catchment]['paths']['gis_catchment_boundary'],
    target_mncat=config[catchment]['target_mncat'],
    catchment=catchment,
    polygon_output_path=config[catchment]['paths']['gis_catchment_dir']
)

# Build catchment mesh
mesh_nodes_table, mesh_nodes_gdf, mesh_cells_gdf_polygons, catchment_polygon = build_mesh(
    shape_filepath=config[catchment]['paths']['gis_catchment_dir'],
    output_path=config[catchment]['paths']['mesh_nodes_output'],
    catchment=catchment,
    grid_resolution=config[catchment]['preprocessing']['graph_construction']['grid_resolution']
)

logger.info(f"Pipeline step 'Build Mesh' complete for {catchment} catchment.")

INFO - Eden and Esk boundary polygon(s) extracted from England data.
data/01_raw/eden/gis/eden_catchment_boundary/eden_combined_boundary.geojson
INFO - Combined Eden and Esk boundary saved to: data/01_raw/eden/gis/eden_catchment_boundary/eden_combined_boundary.geojson
INFO - BUILD_MESH: Starting mesh construction with input: data/01_raw/eden/gis/eden_catchment_boundary/eden_combined_boundary.geojson and resolution: 1000m

INFO - Finding eden catchment spatial boundaries...

INFO - Loading  boundary from: data/01_raw/eden/gis/eden_catchment_boundary/eden_combined_boundary.geojson
INFO - Single polygon found in the catchment boundary.
INFO - Catchment bounding box: min_x=325093.81960000005, min_y=495490.0, max_x=389010.0, max_y=590440.0

INFO - Aligned minx: 325000.0, miny: 495000.0, maxx: 390000.0, maxy: 591000.0
INFO - Number of x-coordinates (bottom-left): 66
INFO - Number of y-coordinates (bottom-left): 97

INFO - Generated 6402 grid cells (centroids and polygons) within bounding box

In [4]:
# Display catchment mesh as map (interactive based on selection)
mesh_map = plot_interactive_mesh(
    mesh_nodes_gdf=mesh_nodes_gdf,
    catchment_polygon=catchment_polygon,
    map_blue=config['global']['visualisations']['maps']['map_blue'],
    esri=config['global']['visualisations']['maps']['esri'],
    esri_attr=config['global']['visualisations']['maps']['esri_attr'],
    static_output_path=config[catchment]['visualisations']['maps']['static_mesh_map_output'],
    interactive_output_path=config[catchment]['visualisations']['maps']['interactive_mesh_map_output'],
    grid_resolution=config[catchment]['preprocessing']['graph_construction']['grid_resolution'],
    interactive=config['global']['visualisations']['maps']['display_interactive_map']
)

logger.info(f"Pipeline step 'Interactive Mesh Mapping' complete for {catchment} catchment.")

# Display map in notebook
mesh_map

INFO - PLOT_INTERACTIVE_MESH: Plotting catchment mesh overlaid on map.
INFO - Interactive map file saved to: results/figures/eden/maps/interactive_mesh_map_1000.html

INFO - Pipeline step 'Interactive Mesh Mapping' complete for eden catchment.
